In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
import matplotlib.pyplot as plt

import pywt
import pywt.data

In [2]:
import subprocess, os, sys
import numpy as np
import nibabel as nib
from scipy import ndimage

In [3]:
img_ct = '../Test/1/CT_PET_Slices'
img_pt = '../Test/1/PET_Attenuation_Correction_Recon'
roi_fln = img_pt

In [4]:
try:
    roi_img = nib.load(roi_fln + '.nii.gz')
except Exception, e:
    print 'INFO: Cannot find the ROI image in NII.GZ. Trying to load the .NII instead.'
    roi_img = nib.load(roi_fln + '.nii')
roi_dat = roi_img.get_data()
roi_aff = roi_img.affine
roi_hdr = roi_img.header

INFO: Cannot find the ROI image in NII.GZ. Trying to load the .NII instead.


In [5]:
roi_dat.shape

(512, 512, 171)

In [6]:
original = ndimage.interpolation.zoom(roi_dat, zoom = 2)
print(original.shape)

(1024, 1024, 342)


In [7]:
coeffsn = pywt.dwtn(original, 'bior1.3')

In [8]:
dx, dy, dz = coeffsn['dad'].shape
print dx, dy, dz

514 514 173


In [9]:
opt_dat = np.zeros_like(roi_dat)
opt_fln = roi_fln + '_texture.nii.gz'

fig = plt.figure()
for i, key in enumerate(coeffsn.keys()):
    ax = fig.add_subplot(3, 3, i + 1)
    tmp_dat = coeffsn[key][1:dx-1,1:dy-1,1:dz-1]    
    ax.imshow(tmp_dat[:,:,100], origin='image', interpolation="nearest",
              cmap=plt.cm.gray)
    ax.set_title(key, fontsize=12)
    if key != 'aaa':
        opt_dat = opt_dat + np.abs(tmp_dat)
        
fig.suptitle("dwtn coefficients", fontsize=14)
plt.show()
texture = opt_dat / 7.

In [10]:
reconstructed = pywt.idwtn(coeffsn, 'bior1.3')


In [11]:
ax = plt.subplot(1, 3, 1)
ax.imshow(original[:,:,105], interpolation="nearest", cmap=plt.cm.gray)
    
ax = plt.subplot(1, 3, 2)
ax.imshow(reconstructed[:,:,100], interpolation="nearest", cmap=plt.cm.gray)


ax = plt.subplot(1, 3, 3)
ax.imshow(texture[:,:,100], interpolation="nearest", cmap=plt.cm.gray)
    
plt.tight_layout()
plt.show()

In [12]:
opt_img = nib.Nifti1Image(opt_dat, roi_aff, header = roi_hdr)
nib.save(opt_img, opt_fln)